Modèle de King et alia

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pickle

# Define the file path in Google Drive
file_path = '/content/drive/MyDrive/Yannis/all_variables.pkl'
def load_variables(year):
    load_path = f'/content/drive/MyDrive/Yannis/variables_{year}.pkl'
    with open(load_path, 'rb') as file:
        variables = pickle.load(file)
    return variables

# Example usage to load variables for a specific year
year_to_load = 2016
loaded_variables = load_variables(year_to_load)

# Access the loaded variables
Eora_pp_FD = loaded_variables['Eora_pp_FD']
labels_FD = loaded_variables['labels_FD']
Eora_pp_Q = loaded_variables['Eora_pp_Q']
labels_Q = loaded_variables['labels_Q']
Eora_pp_QY = loaded_variables['Eora_pp_QY']
Eora_pp_T = loaded_variables['Eora_pp_T']
labels_T = loaded_variables['labels_T']
Eora_pp_VA = loaded_variables['Eora_pp_VA']
labels_VA = loaded_variables['labels_VA']
Eora_pp_T_matrix = loaded_variables['Eora_pp_T_matrix']
Eora_pp_Q_emissions_matrix = loaded_variables['Eora_pp_Q_emissions_matrix']
xout = loaded_variables['xout']
A = loaded_variables['A']
f = loaded_variables['f']
I = loaded_variables['I']
L = loaded_variables['L']
paths = loaded_variables['paths']
sector_labels = loaded_variables['sector_labels']
total_impact = loaded_variables['total_impact']
centrality = loaded_variables['centrality']
bonacich_centrality = loaded_variables['bonacich_centrality']


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ces variables sont stockées sur un drive, et donc pas besoin de les recalculer. Il est toutefois important de se souvenir que cette architecture du code nous force à bien délimiter les réflexions par années, puisque les variables que l'on load le sont aussi. Donc, il faut souvent définir les années à load avant de faire une erreur.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import pickle

# Define the file path in Google Drive
file_path = '/content/drive/MyDrive/Yannis/all_variables.pkl'
def load_variables(year):
    load_path = f'/content/drive/MyDrive/Yannis/variables_for_{year}.pkl'
    with open(load_path, 'rb') as file:
        variables = pickle.load(file)
    return variables

import numpy as np
import pandas as pd
import networkx as nx
import os

# Step 1: Load Data and Initialize Variables
year_to_load = 2015
loaded_variables = load_variables(year_to_load)

Eora_pp_FD = loaded_variables['Eora_pp_FD']
labels_FD = loaded_variables['labels_FD']
Eora_pp_Q = loaded_variables['Eora_pp_Q']
labels_Q = loaded_variables['labels_Q']
Eora_pp_QY = loaded_variables['Eora_pp_QY']
Eora_pp_T = loaded_variables['Eora_pp_T']
labels_T = loaded_variables['labels_T']
Eora_pp_VA = loaded_variables['Eora_pp_VA']
labels_VA = loaded_variables['labels_VA']
Eora_pp_T_matrix = loaded_variables['Eora_pp_T_matrix']
Eora_pp_Q_emissions_matrix = loaded_variables['Eora_pp_Q_emissions_matrix']
xout = loaded_variables['xout']
A = loaded_variables['A']
f = loaded_variables['f']
I = loaded_variables['I']
L = loaded_variables['L']
paths = loaded_variables['paths']
sector_labels = loaded_variables['sector_labels']
total_impact = loaded_variables['total_impact']
centrality = loaded_variables['centrality']
bonacich_centrality = loaded_variables['bonacich_centrality']

# Ensure alpha is defined
alpha = 1  # or any appropriate value

# Step 2: Compute the Leontief Inverse
sectors = Eora_pp_T.shape[0]
I = np.eye(sectors)
FD_vector = Eora_pp_FD.sum(axis=1).values
Gross_output = np.sum(Eora_pp_T, axis=0) + FD_vector
Gross_output = np.where(Gross_output == 0, 1e-10, xout)  # Avoid division by zero
A = Eora_pp_T / Gross_output
Leontief_Inverse = np.linalg.inv(I - A)
V = Leontief_Inverse

# Step 3: Construct the VA Matrix and Adjacency Matrix
VA_Matrix = Eora_pp_T.copy()
max_value = VA_Matrix.max()
VA_Matrix = np.abs(VA_Matrix / max_value)
adjacency_matrix = VA_Matrix.values
print('Creating graph...')
graph = nx.from_numpy_array(adjacency_matrix, create_using=nx.DiGraph)

# Step 4: Emissions Analysis
# Clean the dataset to only retain the emissions, and not other satellite data
emission_indices = labels_Q[labels_Q.iloc[:, 0].str.contains(r"\(Gg\)") & ~labels_Q.iloc[:, 0].str.contains("Total")].index

# Q first
Eora_pp_Q_df_emissions = Eora_pp_Q.iloc[emission_indices]

# QY second
Eora_pp_QY_df_emissions = Eora_pp_QY.iloc[emission_indices]

# Compute the emissions vector
All_Emissions_DF = pd.concat([Eora_pp_Q_df_emissions, Eora_pp_QY_df_emissions], axis=1)
Ei_all = All_Emissions_DF.sum(axis=1).values
Ei_Production = Ei_all[:sectors]
Ei_Consumption = Ei_all[sectors:]

# Compute the Emissions Intensity Matrix (eta, the letter) using vectorized operations
# Step 2: Construct the Emission Intensity (EI) Matrix for a given year

def loead_sectoral_variables(year):
    load_path = f'/content/drive/MyDrive/Yannis/variables_for_{year}.pkl'
    with open(load_path, 'rb') as file:
        variables = pickle.load(file)
    return variables

def construct_ei_matrix(sectoral_variables, labels_T, total_sectors=4915, year):
      EI_matrix = np.zeros((total_sectors, total_sectors))
      sectoral_variables = load_sectoral_variables(year)

  for (country, sector), variables in sectoral_variables.items():
        index = labels_T[(labels_T.iloc[:, 1] == country) & (labels_T.iloc[:, 3] == sector)].index[0]
        EI_matrix[index, index] = variables['EI']

    return EI_matrix
eta = construct_ei_matrix(loaded_variables, labels_T, year_to_load)

def construct X_ij(sectoral_variables):
  X_ij = np.zeros((total_sectors, total_sectors))

  for (country, sector), variables in sectoral_variables.items():

    X_ij = variables['X']
  return X_ij

X_ij = A * Gross_output  # Ensure X_ij is defined
eta = np.where(X_ij > 0, Ei_Production[:, np.newaxis] / X_ij, 0)

# Initialize the Emissions Vector using vectorized operations
computed_emissions = np.sum(eta * X_ij, axis=1)

# Initialize the Emissions Elasticity Matrix using vectorized operations
epsilon = np.where((X_ij > 0) & (computed_emissions[:, np.newaxis] > 0), (X_ij * eta) / computed_emissions[:, np.newaxis], 0)

# Compute Downstream and Upstream Influence Terms using vectorized operations
Downstream_influence = np.dot(V, epsilon.T)
Upstream_influence = Downstream_influence.T

# Calculate Bonacich centrality
Bonacich_centrality = (1 / sectors) * np.sum(V, axis=1)

# Calculate the Aggregate Emissions Impact for Each Sector using vectorized operations
term1 = Ei_all
term2 = np.sum((Ei_all[:, np.newaxis] * Ei_all / Ei_all) * (Downstream_influence / Bonacich_centrality[:, np.newaxis] + Downstream_influence.T / Bonacich_centrality[:, np.newaxis]), axis=1)
term3 = (Ei_all**2 / np.sum(Ei_all)) * np.sum((epsilon**2) / (alpha * A * Bonacich_centrality[:, np.newaxis]), axis=1)
aggregate_emissions_impact = term1 - term2 - term3

# Create DataFrame with results
results_King_df = pd.DataFrame({
    'Sector': sector_labels,
    'Year': year_to_load,
    'Emissions Intensity': list(eta),
    'Downstream Influence': list(Downstream_influence),
    'Upstream Influence': list(Upstream_influence),
    'Emissions Elasticity': list(epsilon),
    'Bonacich Centrality': Bonacich_centrality,
    'Aggregate Emissions Impact': aggregate_emissions_impact
})

# Save the results to a CSV file
results_King_df.to_csv(f'/content/drive/MyDrive/Yannis/results_King_{year_to_load}.csv', index=False)

# Display the DataFrame
results_King_df.head()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 95)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pickle
import numpy as np
import pandas as pd
import networkx as nx
import os

# Define the file path in Google Drive
file_path = '/content/drive/MyDrive/Yannis/all_variables.pkl'

def load_variables(year):
    load_path = f'/content/drive/MyDrive/Yannis/variables_{year}.pkl'
    with open(load_path, 'rb') as file:
        variables = pickle.load(file)
    return variables

# Load Data and Initialize Variables
year_to_load = 2016
loaded_variables = load_variables(year_to_load)

Eora_pp_FD = loaded_variables['Eora_pp_FD']
labels_FD = loaded_variables['labels_FD']
Eora_pp_Q = loaded_variables['Eora_pp_Q']
labels_Q = loaded_variables['labels_Q']
Eora_pp_QY = loaded_variables['Eora_pp_QY']
Eora_pp_T = loaded_variables['Eora_pp_T']
labels_T = loaded_variables['labels_T']
Eora_pp_VA = loaded_variables['Eora_pp_VA']
labels_VA = loaded_variables['labels_VA']
Eora_pp_T_matrix = loaded_variables['Eora_pp_T_matrix']
Eora_pp_Q_emissions_matrix = loaded_variables['Eora_pp_Q_emissions_matrix']
xout = loaded_variables['xout']
A = loaded_variables['A']
f = loaded_variables['f']
I = loaded_variables['I']
L = loaded_variables['L']
paths = loaded_variables['paths']
sector_labels = loaded_variables['sector_labels']
total_impact = loaded_variables['total_impact']
centrality = loaded_variables['centrality']
bonacich_centrality = loaded_variables['bonacich_centrality']

# Ensure alpha is defined
alpha = 1  # or any appropriate value

# Compute the Leontief Inverse
sectors = Eora_pp_T.shape[0]
I = np.eye(sectors)
Gross_output = np.sum(Eora_pp_T, axis=0)
A = Eora_pp_T / Gross_output
Leontief_Inverse = np.linalg.inv(I - A)
V = Leontief_Inverse

# Construct the VA Matrix and Adjacency Matrix
VA_Matrix = Eora_pp_T.copy()
max_value = VA_Matrix.max()
VA_Matrix = np.abs(VA_Matrix / max_value)
adjacency_matrix = VA_Matrix.values
graph = nx.from_numpy_array(adjacency_matrix, create_using=nx.DiGraph)

# Emissions Analysis
# Clean the dataset to only retain the emissions, and not other satellite data
emission_indices = labels_Q[labels_Q.iloc[:, 0].str.contains(r"\(Gg\)") & ~labels_Q.iloc[:, 0].str.contains("Total")].index

# Q first
Eora_pp_Q_df_emissions = Eora_pp_Q.iloc[emission_indices]

# Assuming Eora_pp_Q is your original Q matrix and emission_indices correctly selects emission-related rows
Eora_pp_Q_emissions = Eora_pp_Q.iloc[emission_indices]
# QY second
Eora_pp_QY_df_emissions = Eora_pp_QY.iloc[emission_indices]

# Compute the emissions vector, sum along the rows (data types) to maintain sector columns
Ei_Production = Eora_pp_Q_df_emissions.sum(axis=0).values  # This sums emission data for each sector
Ei_Consumption = Eora_pp_QY_df_emissions.sum(axis=0).values

# Now Ei_Production should match the number of sectors in A and can be used in calculations
Ei_Production_expanded = np.repeat(Ei_Production[:, np.newaxis], A.shape[1], axis=1)
# Compute the Emissions Intensity Matrix (eta)
X_ij = A * Gross_output  # X_ij calculation might need confirmation that A and Gross_output are numpy arrays
eta = np.where(X_ij > 0, Ei_Production_expanded / X_ij, 0)

# Convert computed_emissions to a numpy array if it's not already
computed_emissions = np.sum(eta * X_ij, axis=1)
if isinstance(computed_emissions, pd.Series):
    computed_emissions = computed_emissions.to_numpy()

# Recalculate eta ensuring it is using numpy arrays only
eta = np.where(X_ij > 0, Ei_Production_expanded / X_ij, 0)

# Initialize the Emissions Elasticity Matrix using vectorized operations
epsilon = np.where((X_ij > 0) & (computed_emissions[:, np.newaxis] > 0), (X_ij * eta) / computed_emissions[:, np.newaxis], 0)

# Downstream and upstream influence calculations
Downstream_influence = np.dot(V, epsilon.T)
Upstream_influence = Downstream_influence.T

#Compute Bonacich Centrality avoiding any division by 0
Bonacich_centrality = np.where(np.sum(V, axis=1) != 0, (1 / sectors) * np.sum(V, axis=1), 0)

# Further calculations for the DataFrame
term2 = np.sum((Ei_all[:, np.newaxis] * Ei_all / total_emissions) * (downstream_influence / Bonacich_centrality[:, np.newaxis] + upstream_influence / Bonacich_centrality[:, np.newaxis]), axis=1)
term3 = (Ei_all ** 2 / total_emissions) * np.sum((epsilon ** 2) / (alpha * A * Bonacich_centrality[:, np.newaxis]), axis=1)
aggregate_emissions_impact = Ei_all - term2 - term3  # Ensure Ei_all and term1 are consistent

# Creating DataFrame with results
results_King_df = pd.DataFrame({
    'Sector': sector_labels,
    'Year': year_to_load,
    'Emissions Intensity': list(eta),
    'Downstream Influence': list(Downstream_influence),
    'Upstream Influence': list(Upstream_influence),
    'Emissions Elasticity': list(epsilon),
    'Bonacich Centrality': list(Bonacich_centrality),
    'Aggregate Emissions Impact': list(aggregate_emissions_impact)
})

# Save the results to a CSV file
results_King_df.to_csv(f'/content/drive/MyDrive/Yannis/results_King_{year_to_load}.csv', index=False)

# Display the DataFrame
results_King_df.head()


Mounted at /content/drive


NameError: name 'Ei_all' is not defined

In [ ]:
# Avoiding division by zero in Bonacich centrality and other ratios
Bonacich_centrality = np.where(np.sum(V, axis=1) != 0, (1 / sectors) * np.sum(V, axis=1), 0)


# RETROSPECTIVE ATTEMPT

In [7]:
from google.colab import drive
drive.mount('/content/drive')

import pickle
import numpy as np
import pandas as pd
import networkx as nx
import os

# Define the file path in Google Drive
file_path = '/content/drive/MyDrive/Yannis/all_variables.pkl'

def load_variables(year):
    load_path = f'/content/drive/MyDrive/Yannis/variables_for_{year}.pkl'
    with open(load_path, 'rb') as file:
        variables = pickle.load(file)
    return variables

# Load Data and Initialize Variables
year_to_load = 2016
loaded_variables = load_variables(year_to_load)

Eora_pp_FD = loaded_variables['Eora_pp_FD']
labels_FD = loaded_variables['labels_FD']
Eora_pp_Q = loaded_variables['Eora_pp_Q']
labels_Q = loaded_variables['labels_Q']
Eora_pp_QY = loaded_variables['Eora_pp_QY']
Eora_pp_T = loaded_variables['Eora_pp_T']
labels_T = loaded_variables['labels_T']
Eora_pp_VA = loaded_variables['Eora_pp_VA']
labels_VA = loaded_variables['labels_VA']
Eora_pp_T_matrix = loaded_variables['Eora_pp_T_matrix']
Eora_pp_Q_emissions_matrix = loaded_variables['Eora_pp_Q_emissions_matrix']
xout = loaded_variables['xout']
A = loaded_variables['A']
f = loaded_variables['f']
I = loaded_variables['I']
L = loaded_variables['L']
paths = loaded_variables['paths']
sector_labels = loaded_variables['sector_labels']
total_impact = loaded_variables['total_impact']
centrality = loaded_variables['centrality']
bonacich_centrality = loaded_variables['bonacich_centrality']

# Ensure alpha is defined
alpha = 1  # or any appropriate value

# Compute the Leontief Inverse
sectors =
I = np.eye(sectors)
FD_vector = Eora_pp_FD.sum(axis=1).values
Gross_output = np.sum(Eora_pp_T, axis=0) + FD_vector
Gross_output = np.where(Gross_output == 0, 1e-10, Gross_output)  # Avoid division by zero
A = Eora_pp_T / Gross_output
Leontief_Inverse = np.linalg.inv(I - A)
V = Leontief_Inverse

# Emissions Analysis
emission_indices = labels_Q[labels_Q.iloc[:, 0].str.contains(r"\(Gg\)") & ~labels_Q.iloc[:, 0].str.contains("Total")].index

# Q first
Eora_pp_Q_df_emissions = Eora_pp_Q.iloc[emission_indices]

# QY second
Eora_pp_QY_df_emissions = Eora_pp_QY.iloc[emission_indices]

# Combine emissions
All_Emissions_DF = pd.concat([Eora_pp_Q_df_emissions, Eora_pp_QY_df_emissions], axis=1)
Ei_all = All_Emissions_DF.sum(axis=1).values
Ei_Production = Ei_all[:sectors]
Ei_Consumption = Ei_all[sectors:]

# Define functions to load sectoral variables and construct matrices

def construct_ei_matrix(Eora_pp_T, Ei_Production, Gross_output):
    # Initialize EI matrix
    total_sectors = Eora_pp_T.shape[0]
    EI_matrix = np.zeros((total_sectors, total_sectors))

    for i in range(total_sectors):  # Loop over countries/sectors
        for j in range(total_sectors):  # Loop over sectors within countries
            if Gross_output[j] > 0:
                EI_matrix[i, j] = Ei_Production[i] / Gross_output[j]

    return EI_matrix

eta = construct_ei_matrix(Eora_pp_T, Ei_Production, Gross_output)

def construct_X_ij(Eora_pp_T, Gross_output):
    total_sectors = Eora_pp_T.shape[0]
    X_ij = np.zeros((total_sectors, total_sectors))

    for i in range(total_sectors):  # Loop over countries/sectors
        for j in range(total_sectors):  # Loop over sectors within countries
            X_ij[i, j] = Eora_pp_T[i, j] * Gross_output[j]

    return X_ij

X_ij = construct_X_ij(Eora_pp_T, Gross_output)

# Initialize the Emissions Elasticity Matrix using vectorized operations
epsilon = np.where((X_ij > 0) & (eta > 0), (X_ij * eta) / (np.sum(X_ij * eta, axis=1))[:, np.newaxis], 0)

# Compute Downstream and Upstream Influence Terms using vectorized operations
Downstream_influence = np.dot(V, epsilon.T)
Upstream_influence = Downstream_influence.T

# Construct the VA Matrix and Adjacency Matrix
VA_Matrix = Eora_pp_T.copy()
max_value = VA_Matrix.max()
VA_Matrix = np.abs(VA_Matrix / max_value)
adjacency_matrix = VA_Matrix.values
print('Creating graph...')
graph = nx.from_numpy_array(adjacency_matrix, create_using=nx.DiGraph)

# Calculate Bonacich centrality
def create_network(A, labels):
    G = nx.DiGraph()
    num_nodes = A.shape[0]
    for i in range(num_nodes):
        for j in range(num_nodes):
            if A[i, j] != 0:
                G.add_edge(labels[i], labels[j], weight=A[i, j])
    return G

def find_alpha_beta(G):
    # Get the adjacency matrix of the graph
    A = nx.to_numpy_array(G)
    # Calculate the largest eigenvalue
    largest_eigenvalue = max(np.linalg.eigvals(A))
    # Define alpha as a fraction of the reciprocal of the largest eigenvalue
    alpha = 0.1 * (1 / largest_eigenvalue)
    # Set beta to 1.0 (standard value)
    beta = 1.0
    return alpha, beta

# Compute centrality measures
def compute_centralities(G):
    eigenvector_centrality = nx.eigenvector_centrality_numpy(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    degree_centrality = nx.degree_centrality(G)
    in_degree_centrality = nx.in_degree_centrality(G)  # For directed graphs
    out_degree_centrality = nx.out_degree_centrality(G)  # For directed graphs
    alpha, beta = find_alpha_beta(G)
    bonacich_centrality = nx.katz_centrality_numpy(G, alpha=alpha, beta=beta)

    return {
        'eigenvector': eigenvector_centrality,
        'betweenness': betweenness_centrality,
        'degree': degree_centrality,
        'in_degree': in_degree_centrality,
        'out_degree': out_degree_centrality,
        'bonacich': bonacich_centrality
    }

# Create directed graph
G = create_network(A, sector_labels)
centrality = compute_centralities(G)

# Calculate the Aggregate Emissions Impact for Each Sector using vectorized operations
term1 = Ei_all
term2 = np.sum((Ei_all[:, np.newaxis] * Ei_all / Ei_all) * (Downstream_influence / centrality['bonacich'][:, np.newaxis] + Downstream_influence.T / centrality['bonacich'][:, np.newaxis]), axis=1)
term3 = (Ei_all**2 / np.sum(Ei_all)) * np.sum((epsilon**2) / (alpha * A * centrality['bonacich'][:, np.newaxis]), axis=1)
aggregate_emissions_impact = term1 - term2 - term3

# Create DataFrame with results
results_King_df = pd.DataFrame({
    'Sector': sector_labels,
    'Year': year_to_load,
    'Emissions Intensity': list(eta),
    'Downstream Influence': list(Downstream_influence),
    'Upstream Influence': list(Upstream_influence),
    'Emissions Elasticity': list(epsilon),
    'Bonacich Centrality': centrality['bonacich'],
    'Aggregate Emissions Impact': aggregate_emissions_impact
})

# Save the results to a CSV file
results_King_df.to_csv(f'/content/drive/MyDrive/Yannis/results_King_{year_to_load}.csv', index=False)

# Display the DataFrame
results_King_df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IndexError: index 2047 is out of bounds for axis 0 with size 2047